# PIV

In [ ]:
import cv2
import numpy as np
import os
import csv

# Function that returns the configuration dictionary with all parameters used in processing
def get_config():
    return {
        'video_paths': '/Users/Ricardo/Desktop/Y4 Lab code/All Videos 10',  # Input videos directory
        'output_directory': '/Users/Ricardo/Desktop/Y4 Lab code/PIV_2/videos',  # Output processed videos directory
        'csv_output_directory': '/Users/Ricardo/Desktop/Y4 Lab code/PIV_2/csv',  # Output CSV directory
        'start_frame': 11,  # Starting frame index for processing
        'end_frame': 300,   # Ending frame index for processing
        'window_size': (200, 200),  # Size of interrogation window for PIV
        'x_range': (0, 200),  # X-axis range for splitting windows
        'y_ranges': [(220, 420), (420, 620), (620, 820)],  # Multiple Y ranges for splitting windows
        'gaussian_blur_kernel_size': (13, 13),  # Kernel size for Gaussian blur applied to frames
        'arrow_params': {'color': (0, 0, 0), 'thickness': 2, 'tip_length': 0.1},  # Arrow drawing params
        'video_codec': 'MJPG',  # Codec for output video
        'target_height': 1080,  # Target resolution height (not used in current version)
        'scale_factor': 50,  # Scale factor applied to displacement vectors
        'heatmap_colormap': cv2.COLORMAP_JET,  # Colour map for heatmap visualisation
        'time_averaging_window_size': 1  # Number of frames used for temporal averaging
    }

# Recursively search directory for video files and group them by subfolder
def get_video_paths(input_directory):
    video_paths = {}
    for subdir, _, files in os.walk(input_directory):
        video_files = [filename for filename in files if filename.endswith('.mp4')]
        if video_files:
            folder_name = os.path.basename(subdir)
            video_paths[folder_name] = [os.path.join(subdir, filename) for filename in video_files]
    return video_paths

# Split a frame into smaller windows of given size, restricted to x and y ranges
def split_window(frame, window_size, x_range, y_range):
    return [((x + window_size[0] // 2, y + window_size[1] // 2), 
             frame[y:y + window_size[1], x:x + window_size[0]])
            for x in range(x_range[0], x_range[1], window_size[0])
            for y in range(y_range[0], y_range[1], window_size[1])
            if frame[y:y + window_size[1], x:x + window_size[0]].shape == window_size]

# Perform FFT-based cross-correlation between two interrogation windows
def fft_correlate_images(w1, w2):
    return np.abs(np.fft.fftshift(np.fft.ifft2(np.fft.fft2(w2) * np.conj(np.fft.fft2(w1)))))

# Subpixel peak estimation using quadratic interpolation around the correlation peak
def subpixel_peak_position_quad(shift, max_pos):
    y0, x0 = max_pos
    if x0 <= 0 or x0 >= shift.shape[1] - 1 or y0 <= 0 or y0 >= shift.shape[0] - 1:
        return np.array([y0, x0])
    # Extract correlation values around the peak
    Cx_1, Cx0, Cx1 = shift[y0, x0-1], shift[y0, x0], shift[y0, x0+1]
    Cy_1, Cy0, Cy1 = shift[y0-1, x0], shift[y0, x0], shift[y0+1, x0]
    # Quadratic subpixel displacement estimation
    dx = (Cx1 - Cx_1) / (2 * (2 * Cx0 - Cx_1 - Cx1)) if 2 * (2 * Cx0 - Cx_1 - Cx1) != 0 else 0
    dy = (Cy1 - Cy_1) / (2 * (2 * Cy0 - Cy_1 - Cy1)) if 2 * (2 * Cy0 - Cy_1 - Cy1) != 0 else 0
    return np.array([y0 + dy, x0 + dx])

# Draw an arrow representing displacement vector on a frame
def draw_arrow_on_frame(frame, x, y, u, v, arrow_params):
    cv2.arrowedLine(frame, (int(x), int(y)), (int(x + u), int(y + v)),
                    arrow_params['color'], arrow_params['thickness'],
                    tipLength=arrow_params['tip_length'])

# Apply heatmap colour mapping to a frame
def apply_heatmap(frame, colormap):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    norm = cv2.normalize(gray, None, 0, 255, cv2.NORM_MINMAX)
    heatmap = cv2.applyColorMap(norm, colormap)
    return heatmap

# Process all videos in a folder together, stacking them into a composite video with time-averaging
def process_stacked_video(config, folder_video_paths, subdir_name):
    # Open all videos in the folder
    caps = [cv2.VideoCapture(path) for path in folder_video_paths]
    caps = [cap for cap in caps if cap.isOpened()]
    if not caps:
        return

    # Determine the minimum available number of frames across all videos
    total_frames = min([int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) for cap in caps])
    end_frame = config['end_frame'] if config['end_frame'] < total_frames else total_frames
    start_frame = config['start_frame']

    # Move to the start frame for each video
    for cap in caps:
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    # Determine frame dimensions (assume all videos are the same size)
    w = int(caps[0].get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(caps[0].get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Prepare output video writer and directories
    output_video_name = f"{subdir_name}_stacked.avi"
    fps = caps[0].get(cv2.CAP_PROP_FPS)
    output_video_dir = os.path.join(config['output_directory'], subdir_name)
    os.makedirs(output_video_dir, exist_ok=True)
    output_csv_dir = os.path.join(config['csv_output_directory'], subdir_name)
    os.makedirs(output_csv_dir, exist_ok=True)
    fourcc = cv2.VideoWriter_fourcc(*config['video_codec'])
    out_video = cv2.VideoWriter(os.path.join(output_video_dir, output_video_name), fourcc, fps, (w, h))

    csv_output_path = os.path.join(output_csv_dir, f"{subdir_name}_stacked.csv")
    
    # Buffers for temporal processing:
    # 1. composite_buffer: used for time-averaging across several frames (displacement calculation)
    # 2. raw_buffer: stores the un-averaged composite frames (for display/visualisation)
    composite_buffer = []
    raw_buffer = []
    
    with open(csv_output_path, 'w', newline="") as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['frame_index', 'x', 'y', 'u', 'v'])  # Header row
        
        # Read the first composite frame by averaging across all videos
        frames = []
        for cap in caps:
            ret, frame = cap.read()
            if not ret:
                break
            frames.append(frame)
        if len(frames) != len(caps):
            return
        composite = np.mean(frames, axis=0).astype(np.uint8)
        composite_buffer.append(composite)
        raw_buffer.append(composite)
        
        # Initialise variables for previous averaged composite and raw display frame
        avg_composite_prev = None
        raw_vis_prev = None
        fidx = start_frame + 1  # start from the next frame

        # Process frames until end_frame
        while fidx < end_frame:
            # Read next composite frame (average of all videos at the same index)
            frames_next = []
            for cap in caps:
                ret, frame = cap.read()
                if not ret:
                    break
                frames_next.append(frame)
            if len(frames_next) != len(caps):
                break
            composite = np.mean(frames_next, axis=0).astype(np.uint8)
            
            # Append to buffers
            composite_buffer.append(composite)
            raw_buffer.append(composite)
            # Keep buffers within window size
            if len(composite_buffer) > config['time_averaging_window_size']:
                composite_buffer.pop(0)
            if len(raw_buffer) > config['time_averaging_window_size']:
                raw_buffer.pop(0)
            
            # Skip until buffer is fully filled
            if len(composite_buffer) < config['time_averaging_window_size']:
                fidx += 1
                continue
            
            # Compute time-averaged composite for displacement calculation
            avg_composite_current = np.mean(np.array(composite_buffer), axis=0).astype(np.uint8)
            
            # If first time, initialise and continue
            if avg_composite_prev is None:
                avg_composite_prev = avg_composite_current
                raw_vis_prev = raw_buffer[-1]  # Use newest raw frame for visualisation
                fidx += 1
                continue
            
            # Preprocess both averaged composite frames (Gaussian blur + grayscale)
            f1g = cv2.GaussianBlur(cv2.cvtColor(avg_composite_prev, cv2.COLOR_BGR2GRAY),
                                   config['gaussian_blur_kernel_size'], 0)
            f2g = cv2.GaussianBlur(cv2.cvtColor(avg_composite_current, cv2.COLOR_BGR2GRAY),
                                   config['gaussian_blur_kernel_size'], 0)
            
            # Split into interrogation windows
            w1, w2 = [], []
            for yr in config['y_ranges']:
                w1.extend(split_window(f1g, config['window_size'], config['x_range'], yr))
                w2.extend(split_window(f2g, config['window_size'], config['x_range'], yr))
            
            # Compute displacement vectors
            averaged_displacements = []
            for ((cx, cy), window1), ((_, _), window2) in zip(w1, w2):
                shift = fft_correlate_images(window1, window2)
                mxp = np.unravel_index(np.argmax(shift), shift.shape)
                sp = subpixel_peak_position_quad(shift, mxp)
                dx = sp[1] - shift.shape[1] // 2
                dy = sp[0] - shift.shape[0] // 2
                averaged_displacements.append(((cx, cy), (dx * config['scale_factor'], dy * config['scale_factor'])))
            
            # Visualisation: apply heatmap to raw (not averaged) composite frame
            heat_frame = apply_heatmap(raw_vis_prev, config['heatmap_colormap'])
            for (x, y), (u, v) in averaged_displacements:
                draw_arrow_on_frame(heat_frame, x, y, u, v, config['arrow_params'])
                csv_writer.writerow([fidx, x, y, u, v])
            
            # Save processed frame
            out_video.write(heat_frame)
            
            # Update previous references
            avg_composite_prev = avg_composite_current
            raw_vis_prev = raw_buffer[-1]
            fidx += 1

    for cap in caps:
        cap.release()
    out_video.release()

# Main script: prepares directories, loads videos, and processes each folder of videos
def main():
    config = get_config()
    os.makedirs(config['output_directory'], exist_ok=True)
    os.makedirs(config['csv_output_directory'], exist_ok=True)
    video_paths = get_video_paths(config['video_paths'])
    
    if video_paths:
        for subdir_name, video_files in video_paths.items():
            # Process each folder into a stacked video with time-averaged composites
            process_stacked_video(config, video_files, subdir_name)

if __name__ == "__main__":
    main()
